# Image Processing Laboratory Notebooks <img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" alt="EPFL_logo" style="padding-right:10px;height:40px;float:left">

---

This Jupyter Notebook is part of a series of computer laboratories designed
to teach image-processing programming. They are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-511 Image Processing I**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2024.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Yan Liu](mailto:yan.liu@epfl.ch),
    [Zhiyuan Hu](mailto:zhiyuan.hu@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).

---

# Lab 2.2: Filtering Applications
**Released**: Thursday, November 7, 2024

**Submission deadline**: Monday, November 18, 2024, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=522)

**Grade weight**: Lab 2 (18 points), 10 % of the overall grade

<div class="alert alert-danger">
<b>Important:</b> don't forget to write down your name and SCIPER!
</div>

## Student Name: 
## SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## Imports
In the next cells we import the libraries and load the images that we will use throughout the lab.

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import skimage
import scipy
from skimage import io
from interactive_kit import imviewer as viewer

# Load images to be used in this exercise 
bikesgray = io.imread('images/bikesgray.tif').astype('float64')
camera = io.imread('images/camera-16bits.tif').astype('float64')
spots = io.imread('images/spots.tif').astype('float64')

In [ ]:
%use javascript
%get bikesgray camera spots
// import IPLabImageAccess as Image
var Image = require('image-access')

# Filtering applications (9 points)

After the [first part](./1_Filtering.ipynb) of the lab, we expect you to feel comfortable with the basics of filtering. In this part we will look in detail at the implementation of a Gaussian filter, as well as some of its direct applications. Gaussian filters are known to be near-optimal smoothing filters, and represent perhaps the most used preprocessing step in image processing to improve robustness in a workflow and to denoise images.

### Visualize images
Get familiar with the images that you are going to use by running the next cell.

In [ ]:
%use sos
image_list = [bikesgray, camera, spots]
imgs_viewer = viewer(image_list, widgets=True, hist=True)

# 1. Gaussian filter (4 points)

An [isotropic 2D Gaussian](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) is represented by its impulse response,

$$h_\sigma(x,y) = \frac{1}{2\pi\sigma^2}\exp\left(-\frac{x^2 + y^2}{2\sigma^2}\right) \overset{\mathrm{separability}}{=} \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{x^2}{2\sigma^2}\right) \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{ y^2}{2\sigma^2}\right) \,,$$

where $\sigma$ is the standard deviation and controls the smoothing strength.

In this section, you will implement a 2D Gaussian filter with impulse response $h_{\sigma}[m,n]$, which discretizes $h_\sigma(x,y)$ between $[-\lceil3\sigma\rceil,\lceil 3\sigma\rceil]$ in $x$ and $y$. Here, $\lceil x \rceil$ refers to the smallest integer larger than a given $x\in\mathbb{R}$. 

Choose the size of the filter to be $N = 2\lceil 3\sigma \rceil+1$ (hence, $N$ is always odd), and **ensure the impulse response adds up to $1$**, using appropriate normalization.

## 1.A. Implementation of a 2D Gaussian filter (3 points)

For **3 points**, implement the function `gaussian(img, sigma)` that convolves an image with a Gaussian filter using a **separable implementation** in JavaScript. Take advantage of the `filter1D` function from Section [1.B.](./1_Filtering.ipynb#1.B.-Separable-implementation-(2-points)) that we have copied here for you in the next cell.

<div class="alert alert-success">
    <b>Hint: </b>
    <ul>
    <li>You can use <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Math">the <code>Math</code> library</a> to access different mathematical functions.</li>
    <li>The first argument to the <code>Image</code> constructor is the <b>height</b> and the second is the <b>width</b>: <code>new Image(height, width)</code> or <code>new Image([height, width])</code>. Feel free to go back to <a href="../0_Introductory_lab/Introductory.ipynb#3.-Javascript-image-access-class-(2-points)">Lab 0: Introduction</a> to review the basic usage of the ImageAccess class.</li>
    </ul>
</div>

In [ ]:
%use javascript
// function that performs a gaussian filter with sigma on img
function gaussian(img, sigma){
    var output = new Image(img.shape());
    
    // Define normalized mask
    // YOUR CODE HERE
    
    // Filter using separable implementation (hint: your mask should be 1D)
    // You can use the filter1D function defined below
    // YOUR CODE HERE
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1);
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        output.setPixel(x, 0, val);
    }
    return output
}

We have designed a quick test for you to evaluate your method, applying it to a $5 \times 5$ impulse image. Run the following cell and check that your output has all the desired properties of a Gaussian.

In [ ]:
%use javascript

var impulse = new Image([[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]);
var impulse_gaussian = gaussian(impulse, 0.5);

// look at result, verify that it has the properties of a Gaussian
console.log('Your impulse Gaussian:\n' + impulse_gaussian.visualize());

// this assertion checks proper behaviour: that the center is the maximum, and that two pixels in equivalent positions have the same values.
if(impulse_gaussian.getPixel(2, 2) < impulse_gaussian.getPixel(2, 3) || impulse_gaussian.getPixel(2, 1) !== impulse_gaussian.getPixel(2, 3)){
    console.log('WARNING!!!\nThere are still some mistakes with your implementation! Look at the sanity checks to understand the mistakes');
}else{
    console.log('The symmetry of the Gaussian seems good.');
}

// check normalization
var sum = 0
for(var x = 0; x < impulse_gaussian.nx; x++){
    for (var y = 0; y < impulse_gaussian.ny; y++){
        sum += impulse_gaussian.getPixel(x, y);
    }
}
if(Math.abs(sum - 1) > 1e-5){
    console.log("WARNING!!\nNormalization not correct");
}else{
    console.log("Well done! The output sums up to approximately 1.");
}

We will apply your Gaussian filter to the image `bikesgray` using various values of $\sigma$. Observe the changes in the mean and standard deviation by checking the statistics box in the `viewer`, or by using the functions `np.mean` and `np.std`. 

Run and modify the two following cells to apply Gaussian filters with different $\sigma$ values to `bikesgray` and view the result. Then, answer the two multiple choice questions.

<div class="alert alert-danger">
<b>Important:</b> don't forget to change $\sigma$ to a normal (small) value after your exploration!
</div>

In [ ]:
%use javascript
%put bikesgray_gaussian1 bikesgray_gaussian5

// apply filter to Image object. To try different sigma values, change the variables or declare more. 
var bikesgray_gaussian1 = gaussian(new Image(bikesgray), 1).toArray()
var bikesgray_gaussian5 = gaussian(new Image(bikesgray), 5).toArray()

In [ ]:
%use sos

# Declare parameters for ImageViewer. If you want to visualize more sigma values, update the previous cell and these lists accordingly
image_list_blur = [bikesgray, bikesgray_gaussian1, bikesgray_gaussian5]
title_list_blur = ['Original', 'Sigma: 1', 'Sigma: 5']
for i in range(len(image_list_blur)):
    image_list_blur[i] = np.array(image_list_blur[i])
plt.close('all')
blurred_bikesgray_viewer = viewer(image_list_blur, title=title_list_blur, hist=True)

### Multiple Choice Question

After modifying the two cells above and visualizing the results, answer the next two questions (worth **0.5 points** each).

* Q1: How will the Fourier transform of an image change after applying a Gaussian filter?
    1. It will have lower values for higher frequencies.
    2. It will have higher values for higher frequencies.
    3. It will have lower values for lower frequencies.
    4. It will not change.


* Q2: What will be the output image when $\sigma\rightarrow \infty$? What type of filter would that be?
    1. An image equal to the original. It would be an all-pass filter.
    2. A constant image. It would be a high-pass filter.
    3. A 2D Gaussian. It would be a band-pass filter.
    4. A constant image. It would be a low-pass filter.

Modify the variables `answer_one` and `answer_two` in the next cell to match your choices. The second and third cells are for you to make sure that your answer is in the valid range (they should not raise any error).

In [ ]:
%use sos
# Modify these variables
answer_one = None
answer_two = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity test
if not answer_one in [1, 2, 3, 4, 5]:
    print('WARNING!\nAnswer one of 1, 2, 3, 4 or 5.')

In [ ]:
%use sos
# Sanity test
if not answer_two in [1, 2, 3, 4]:
    print('WARNING!\nAnswer one of 1, 2, 3 or 4.')

## 1.B. Gaussian filter in Python (1 point)

There are several implementations of Gaussian filters in Python. In this section, we will use the `scikit-image` implementation [`skimage.filters.gaussian`](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian). Here is an example how to use it:

```python
output = skimage.filters.gaussian(input, sigma=10, mode='reflect', truncate=3, preserve_range=True)
```
The above line specifies that we apply on the `input` image a Gaussian filter with $\sigma=10$, reflective boundary condition, truncated to $3\sigma$ range and keep the original range of values of `input`.


In the next cells, we will apply it to the image `bikesgray` and compare it to your implementation. 

In [ ]:
%use sos

bikesgray_gaussian_skimage = skimage.filters.gaussian(bikesgray, sigma=10 , mode='reflect', truncate=3, preserve_range=True)
gaussian_viewer = viewer(bikesgray_gaussian_skimage)

Now, we will compare it to your implementation in JavaScript to make sure that they are equivalent (up to errors on the order of $10^{-14}$). For this, we call the `gaussian` function you implemented with the image `bikesgray`, also for $\sigma = 10$.

If the images are not the same, an `ImageViewer` will show the differing regions in red to help identify mistakes.  

In [ ]:
%use javascript
%put bikesgray_gaussian10

var bikesgray_gaussian10 = gaussian(new Image(bikesgray), 10).toArray()

In [ ]:
%use sos

bikesgray_gaussian10 = np.array(bikesgray_gaussian10)
image_list = [bikesgray_gaussian10, bikesgray_gaussian_skimage, np.abs(bikesgray_gaussian_skimage - bikesgray_gaussian10)]
title_list = ['JS', 'Skimage', 'Difference']
plt.close('all')
if not np.allclose(bikesgray_gaussian10, bikesgray_gaussian_skimage):
    print('The results of your Gaussian filter do not match Skimage results! Look at the red areas in the viewer to see where you might have gone wrong.')
    skimage_gaussian_viewer = viewer([bikesgray_gaussian10, bikesgray_gaussian_skimage], title=['JS', 'Skimage'], widgets=True, compare=True)
else :
    print('Seems like your Gaussian filter is correct!')

# 2. Application: Spot detector (5 points)

Now, we will apply Gaussian filtering to create a *spot detector* using the Difference of Gaussians (DoG) filter, which is an approximation of the Laplacian-of-Gaussian (LoG) filter. We will perform these tasks using **only Python libraries**.

To detect spots, we will compute the local maximum on the output of the DoG filter. In the upcoming exercises, we will work with an image called `spots`. Run the next cell to visualize the image.

<div class = 'alert alert-success'>
<b>Note</b>: At first, it may seem tempting to use a simple thresholding approach to detect <i><u>bright</u></i> spots, given their higher intensity compared to the rest of the image. However, by hovering your mouse around the image and examining the pixel values, you will quickly realize that there exists no such threshold. This is precisely why we need to use more sophisiticated techniques. Moreover, the <i>spot detector</i> you will code is robust to noise, and to changing pixel values accross images. 

In [ ]:
%use sos
plt.close('all')
spots_vis = viewer(spots)

## 2.A. Difference of Gaussians (2 points)

The DoG is constructed from the subtraction of two Gaussian functions, i.e., $\mathrm{DoG}(x) = h_{\sigma_{1}}(x) - h_{\sigma_2}(x)$. It is usually parametrised only by $\sigma_1$, and $\sigma_2$ is chosen as $\sigma_2 = \sqrt{2}\sigma_1$. 

Experiment with the value of $\sigma_1$ in the next cell to see the kind of profile generated by this filter in 1D.

In [ ]:
%use sos
# Choose sigmas
sigma_slider = widgets.FloatSlider(value=0.5, min=0.5, max=5, description=r'$\sigma_1$')

# Initialize figure
plt.close('all')
fig, ax = plt.subplots(1, 1, num=f"Difference of Gaussians filter in 1D - SCIPER: {uid}", figsize=[10,4])
ax.plot(0, 0, 0, 0, 0, 0);
ax.set_xlabel(r"$x$"); plt.legend([r"$h_{\sigma_1}(x)$", r"$h_{\sigma_2}(x)$", r"$\mathrm{DoG}_{\sigma_1}(x)$"]);
ax.set_xlim([-20, 20]); ax.set_ylim([-0.1, 0.8])
ax.grid(); fig.tight_layout()
 
# Plotting function - Callback for slider
def dog_1d(change):
    # Get value of sigma, initialize variables of interest and clear axes
    sigma_1 = change.new
    sigma_2 = sigma_1*np.sqrt(2)
    # Update plot
    x = np.arange(-3*sigma_2, (3+6./100)*sigma_2, 6*sigma_2/100)
    ax.lines[0].set_data(x, scipy.stats.norm(scale=sigma_1).pdf(x))
    ax.lines[1].set_data(x, scipy.stats.norm(scale=sigma_2).pdf(x))
    ax.lines[2].set_data(x, scipy.stats.norm(scale=sigma_1).pdf(x) - scipy.stats.norm(scale=sigma_2).pdf(x));

sigma_slider.observe(dog_1d, 'value')
sigma_slider.value = 1
display(sigma_slider)

For **1 point**, modify the next cell and write the function `dog`, that takes as input an image and the value of $\sigma_1$, then outputs the normalized DoG of this image so that its intensity is in the range$[0,1]$. **You should hardcode the value of $\sigma_2 = \sigma_1\sqrt{2}$ inside the function**. 


<div class = 'alert alert-info'>
    <b>Note</b>: Check the documentation of the <code>skimage.filters.gaussian</code> function <a href='https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian'>here</a>.  <b>Make sure you use the correct parameters</b>, like <code>preserve_range = True</code> option, truncating the filter at $3\sigma$ (so $N = 2\lceil 3\sigma \rceil+1$), and using <code>'reflect'</code> or equivalent boundary conditions.
</div>

Complete the function `dog` in the next cell, where we have also included an initial sanity check.

In [ ]:
%use sos

def dog(image, sigma_1):
    output = np.copy(image)
    
    # Apply the DoG filter to image
    # YOUR CODE HERE
    
    return output

err_message = "Remember to normalize the output so that it spans the range [0,1]."
assert dog(spots, 1).max() == 1, err_message
assert dog(spots, 1).min() == 0, err_message

In the next two cells, you will visualize the results of your function for different $\sigma_1$ values. Go to the menu `Extra Widgets`, where you can find a slider to modify the value of $\sigma_1$.  

In [ ]:
%use sos
# Define sliders and button
sigma_slider = widgets.FloatSlider(value=1, min=0.5, max=10.0, step=0.5, description='\u03c3\u2081:')
button = widgets.Button(description='Apply DoG')
# Define callback function
def button_dog(image):
    sigma = sigma_slider.value
    image = dog(image, sigma)
    return image

In [ ]:
%use sos

plt.close("all")
dog_viewer = viewer(spots, title = "DoG Spots", new_widgets = [sigma_slider, button], callbacks=[button_dog], widgets=True, normalize=True)

### Multiple choice question

* Q1: What type of filter is the DoG?

    1. Low-pass  
    2. Band-pass  
    3. High-pass  


* Q2: Which $\sigma$ would you choose to highlight the spots?

    1. 1.5  
    2. 5  
    3. 10 


Modify the variables `answer_one` and `answer_two` in the next cell to your choices.

In [ ]:
%use sos
# Modify these variables
answer_one = None
answer_two = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer_one in [1, 2, 3]:
    print('WARNING!\nAnswer one of 1, 2 or 3.')
assert answer_one in [1, 2, 3], 'Choose one of 1, 2 or 3.'

In [ ]:
%use sos
# Sanity check
if not answer_two in [1, 2, 3]:
    print('WARNING!\nAnswer one of 1, 2 or 3.')

## 2.B. Local maxima (1 point)

Now you will write the function `local_max(img, T)` that returns a binary image. In a $3\times 3$ neighborhood, this function will set the pixels which are a local maximum to $255$ and rest to $0$. A local maximum in a $3\times 3$ neighborhood is a pixel that has a value strictly greater than its 8 closest neighbors (8-connected) and is strictly greater than a threshold $T$ (specified between $0$ and $1$, relative to the maximum of the image).

<div class="alert alert-info">
    <b>Hint:</b> 8-connected pixels are neighbors to every pixel that touches one of their edges or corners.<br>
    <table><tr>
    <td>
      <p align="center" style="padding: 0px">
        <img src="images/8_connectivity.jpg" alt="8-connectivity" width="100px"><br>
      </p>
    </td>
    </tr></table>

The function <code>skimage.feature.peak_local_max</code> (<a href = 'https://scikit-image.org/docs/0.7.0/api/skimage.feature.peak'>see documentation here</a>) is useful. Remember you don't need and shouldn't use for-loops in this exercise!

</div>
    
For **1 point**, modify the next cell to define your function.

In [ ]:
%use sos

# Function that computes the local max in a 3x3 nbh
def local_max(img, T):
    output = np.zeros(img.shape)
    
    # Apply the local maxima
    # YOUR CODE HERE
    
    return output

Run the next cell for a quick test on your function. We test  that your image applied to `camera` with a threshold $T = 0.5$ detects exactly the four maximum points of the image, as it should. If the assertion raises no error, your function is most probably correct.

In [ ]:
%use sos

if np.count_nonzero(local_max(camera, 0.5)) in [4, 5]:
    print('Congratulations! Your function passed this sanity check.')
else :
    print('WARNING!!\nYour function is not working on the image `cameras` as it should.')

Now you are going to see the effect of this function through a slider in `Extra Widgets`. Run the next cell to test it on the image `camera`. 

Then modify the next cell to look at the result of your function applied to the image `spots`. Is function able to detect the **6 spots**?  

<div class = 'alert alert-info'>
<b>Note</b>: In the image <code>camera</code>, you might notice a somewhat strange behaviour with the bright pixel located at the bottom of the image. Do you know why is this happening? If you don't, have a closer look at the documentation of <code>feature.peak_local_max</code>! 
</div>

In [ ]:
%use sos

threshold_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='T:')
button = widgets.Button(description='Apply Local Maxima')

def button_local_max(image):
    t = threshold_slider.value
    image = local_max(image, t)
    return image

local_max_viewer = viewer(camera, title="Local Maxima", new_widgets=[threshold_slider, button], callbacks=[button_local_max], widgets=True)

## 2.C. Spot detector (2 points)

For **1 point**, implement the method `spot_detector(img, sigma, T)`, where you use your previous two functions to detect spots. In other words, apply the detection of local maxima on the output of the DoG filter.

In [ ]:
%use sos

# Function that detects spots in img, using sigma and a threshold T
def spot_detector(img, sigma, T):
    output = None
    
    # YOUR CODE HERE
    
    return output

Run the next cell for a quick test on your function.

In [ ]:
%use sos

if not np.count_nonzero(spot_detector(spots, 1, 0.3)) == 6:
    print('WARNING!!!\nYour function is not yet correct. First make sure that `dog` and `local_max` are.')
else :
    print('Congratulations! Your spot detector seems to be correct.')

Now, let's apply your function to the image `spots` inside an `ImageViewer`, using two sliders for the values of $\sigma_1$ and $T$. Run the following cell, and play with these values (access the sliders through the button `Extra Widgets`). Explore the results also on other images.

In [ ]:
%use sos

# Define sliders
sigma_slider = widgets.FloatSlider(value=5, min=0.5, max=10.0, step=0.5, description="\u03c3\u2081:")
t_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='T:')
button = widgets.Button(description='Apply Spot Detection')

# Define callback function
def button_spot_detection(image):
    sigma = sigma_slider.value
    t = t_slider.value
    image = spot_detector(image, sigma, t)
    contours, _ = skimage.measure.find_contours(image.astype(np.uint8))
    print(f'Detected {len(contours):4} spots.', end='\r')
    return image

plt.close("all")
spot_detector_viewer = viewer(spots, title="Spot Detector", new_widgets=[sigma_slider, t_slider, button], callbacks=[button_spot_detection], widgets=True, normalize=True )

### Multiple Choice Question

What pair of parameters will give you exactly 6 spots? If there are more than one, try to select the most reasonable one. 

1. $\sigma_1 = 10$ and $T = 0.2$
2. $\sigma_1 = 5$ and $T = 0.6$
3. $\sigma_1 = 5$ and $T = 0.2$
4. $\sigma_1 = 1$ and $T = 0.3$

Modify the variable answer in the next cell to reflect your choice. Run the last cell to check that your answer is valid.

In [ ]:
%use sos
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity check
if not answer in [1, 2, 3, 4]:
    print('WARNING!\nAnswer one of 1, 2, 3 or 4.')

<div class="alert alert-success">
    
<p><b>Congratulations on finishing Lab 2!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1111434">Moodle</a>, in a zip file together with the first part of this lab.
</p>
</div>

* Keep the name of the notebook as: *2_Filtering_Applications.ipynb*,
* Name the zip file: *Filtering_lab.zip*.